In [1]:
! python3 --version

Python 3.12.3


In [ ]:
! pip install transformers torch accelerate

In [ ]:
! pip install python_dotenv

In [7]:
! pip freeze > requirements.txt

### Huggingface login

In [ ]:
# manual
# Do this from terminal so you can paste the token when requested
! huggingface-cli login

In [5]:
# Programmatically
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()

access_token_read = os.environ.get('HUGGINGFACE_TOKEN')

login(token = access_token_read)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/vrbabu/.cache/huggingface/token
Login successful


In [3]:
# Logout
from huggingface_hub import logout
logout()

Successfully logged out.


/Users/vrbabu/Library/CloudStorage/OneDrive-Personal/Documents/Documents/code/Llama2_with_Hugging_Face_Pipeline/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
! huggingface-cli whoami

Gilgamesh37
